In [1]:
# --- Day 11: Single source of truth for league radar ---

from pathlib import Path
import sys

# Anchor to repo root (.../nba)
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.data_loader import load_default_2024_25
from src.radar import build_league_radar

# Load canonical dataset
df = load_default_2024_25()

# Build league-wide replacement radar
radar = build_league_radar(
    df,
    star_min_threshold=30.0,
    min_games_without=5
)

radar.shape


(1309, 11)

In [2]:
# 🔁 Reload pipeline modules after editing src/
import importlib
import src.metrics
import src.radar

importlib.reload(src.metrics)
importlib.reload(src.radar)

from src.radar import build_league_radar
from src.metrics import add_score_breakdown


In [3]:
cols = ["STAR_OUT","TEAM_NAME","BENEFICIARY","N_WITHOUT","LOW_SAMPLE",
        "DELTA_PTS","DELTA_AST","DELTA_REB","DELTA_MIN","SCORE"]

radar.sort_values("SCORE", ascending=False)[cols].head(15)



,STAR_OUT,TEAM_NAME,BENEFICIARY,N_WITHOUT,LOW_SAMPLE,DELTA_PTS,DELTA_AST,DELTA_REB,DELTA_MIN,SCORE
1192,Trae Young,Hawks,Keaton Wallace,6,False,10.344444,5.977778,3.522222,19.004630,38.849074
1050,Paul George,76ers,Chuma Okeke,5,False,8.800000,2.200000,7.000000,20.500000,38.500000
1209,Trey Murphy III,Pelicans,Keion Brooks Jr.,12,False,9.972222,0.805556,3.416667,21.310303,35.504747
1158,Shai Gilgeous-Alexander,Thunder,Dillon Jones,6,False,7.959770,1.632184,4.333333,21.229524,35.154811
657,Joel Embiid,76ers,Chuma Okeke,6,False,7.166667,1.750000,5.500000,20.500000,34.916667
684,Josh Hart,Knicks,Precious Achiuwa,5,False,8.678481,0.916456,5.625316,17.019231,32.239484
1161,Shai Gilgeous-Alexander,Thunder,Kenrich Williams,5,False,9.776087,1.600000,4.147826,16.656321,32.180234
670,Joel Embiid,76ers,Lonnie Walker IV,21,False,11.761905,2.380952,1.904762,15.063684,31.111303
566,Jalen Williams,Thunder,Branden Carlson,6,False,8.969697,1.007576,3.295455,17.553857,30.826584
169,CJ McCollum,Pelicans,Keion Brooks Jr.,7,False,11.142857,0.642857,3.857143,14.877143,30.520000


In [4]:
REPORTS_DIR = PROJECT_ROOT / "data" / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

csv_out = REPORTS_DIR / "league_replacement_radar_2024_25.csv"
radar.to_csv(csv_out, index=False)

print("Saved:", csv_out)
print("Exists:", csv_out.exists())
print("Bytes:", csv_out.stat().st_size)


Saved: C:\Users\bo_co\ai-sports-competitive-intel\nba\data\reports\league_replacement_radar_2024_25.csv
Exists: True
Bytes: 184179


In [5]:
import pandas as pd
pd.read_csv(csv_out).shape



(1309, 11)

In [6]:
radar[["DELTA_PTS","DELTA_AST","DELTA_REB","DELTA_MIN","SCORE"]].isna().sum()


DELTA_PTS    100
DELTA_AST    100
DELTA_REB    100
DELTA_MIN    121
SCORE        121
dtype: int64

In [8]:
radar = build_league_radar(df, star_min_threshold=30.0, min_games_without=5)
radar = add_score_breakdown(radar)

assert radar["SCORE"].notna().all()
print("OK: SCORE has no NaNs")

radar.sort_values("SCORE", ascending=False).head(10)


OK: SCORE has no NaNs


,STAR_OUT,TEAM_CITY,TEAM_NAME,BENEFICIARY,SCORE,DELTA_PTS,DELTA_AST,DELTA_REB,DELTA_MIN,N_WITHOUT,LOW_SAMPLE
1192,Trae Young,Atlanta,Hawks,Keaton Wallace,38.849074,10.344444,5.977778,3.522222,19.004630,6,False
1050,Paul George,Philadelphia,76ers,Chuma Okeke,38.500000,8.800000,2.200000,7.000000,20.500000,5,False
1209,Trey Murphy III,New Orleans,Pelicans,Keion Brooks Jr.,35.504747,9.972222,0.805556,3.416667,21.310303,12,False
1158,Shai Gilgeous-Alexander,Oklahoma City,Thunder,Dillon Jones,35.154811,7.959770,1.632184,4.333333,21.229524,6,False
657,Joel Embiid,Philadelphia,76ers,Chuma Okeke,34.916667,7.166667,1.750000,5.500000,20.500000,6,False
684,Josh Hart,New York,Knicks,Precious Achiuwa,32.239484,8.678481,0.916456,5.625316,17.019231,5,False
1161,Shai Gilgeous-Alexander,Oklahoma City,Thunder,Kenrich Williams,32.180234,9.776087,1.600000,4.147826,16.656321,5,False
670,Joel Embiid,Philadelphia,76ers,Lonnie Walker IV,31.111303,11.761905,2.380952,1.904762,15.063684,21,False
566,Jalen Williams,Oklahoma City,Thunder,Branden Carlson,30.826584,8.969697,1.007576,3.295455,17.553857,6,False
169,CJ McCollum,New Orleans,Pelicans,Keion Brooks Jr.,30.520000,11.142857,0.642857,3.857143,14.877143,7,False


In [11]:
import pandas as pd
check = pd.read_csv(radar_out2)
print(check.shape)
assert len(check) == len(radar)
assert check["SCORE"].notna().all()
print("OK: saved CSV reads back clean")


(1309, 11)
OK: saved CSV reads back clean
